In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  hidden_nodes = 1024
  lambd = 0.7
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.random.truncated_normal([hidden_nodes, num_labels]))
  hidden_weights = tf.Variable(tf.random.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  hidden_layers = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
  hidden_layers = tf.nn.relu(hidden_layers)
  logits = tf.matmul(hidden_layers,weights)+biases
  l2_regularization = lambd/(2*train_dataset.shape[0])* (tf.nn.l2_loss(weights)+tf.nn.l2_loss(hidden_weights))
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+l2_regularization
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.matmul(tf_valid_dataset,hidden_weights)+hidden_biases
  valid_logits = tf.nn.relu(valid_hidden)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_logits, weights) + biases)
  test_hidden = tf.matmul(tf_test_dataset,hidden_weights)+hidden_biases
  test_logits = tf.nn.relu(test_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(test_logits, weights) + biases)

In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.compat.v1.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.compat.v1.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 335.931183
Training accuracy: 7.5%
Validation accuracy: 45.6%
Loss at step 100: 4.910677
Training accuracy: 91.4%
Validation accuracy: 78.7%
Loss at step 200: 9.213683
Training accuracy: 86.8%
Validation accuracy: 75.3%
Loss at step 300: 0.568888
Training accuracy: 99.8%
Validation accuracy: 78.7%
Loss at step 400: 0.581593
Training accuracy: 99.8%
Validation accuracy: 78.8%
Loss at step 500: 0.647472
Training accuracy: 99.7%
Validation accuracy: 79.1%
Loss at step 600: 0.587381
Training accuracy: 99.7%
Validation accuracy: 79.1%
Loss at step 700: 0.601445
Training accuracy: 99.7%
Validation accuracy: 79.0%
Loss at step 800: 0.597328
Training accuracy: 99.7%
Validation accuracy: 79.0%
Test accuracy: 86.5%


In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.compat.v1.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.compat.v1.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  l2_regularization = lambd/(2*tf_train_dataset.shape[0])* tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+l2_regularization
  
  # Optimizer.
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.compat.v1.Session(graph=graph) as session:
  tf.compat.v1.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.741016
Minibatch accuracy: 12.5%
Validation accuracy: 12.6%
Minibatch loss at step 500: 2.444001
Minibatch accuracy: 82.8%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 1.137426
Minibatch accuracy: 82.0%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 0.593751
Minibatch accuracy: 83.6%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.601371
Minibatch accuracy: 89.1%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 0.721581
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.714895
Minibatch accuracy: 83.6%
Validation accuracy: 81.9%
Test accuracy: 89.1%


In [9]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.compat.v1.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  hidden_layers = 1024
  
  # Variables.    
  weights = tf.Variable(
  tf.random.truncated_normal([hidden_nodes, num_labels]))
  hidden_weights = tf.Variable(tf.random.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  hidden_layers = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
  hidden_layers = tf.nn.relu(hidden_layers)
  logits = tf.matmul(hidden_layers,weights)+biases
  l2_regularization = lambd/(2*train_dataset.shape[0])* (tf.nn.l2_loss(weights)+tf.nn.l2_loss(hidden_weights))
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+l2_regularization

  # Optimizer.
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.matmul(tf_valid_dataset,hidden_weights)+hidden_biases
  valid_logits = tf.nn.relu(valid_hidden)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_logits, weights) + biases)
  test_hidden = tf.matmul(tf_test_dataset,hidden_weights)+hidden_biases
  test_logits = tf.nn.relu(test_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(test_logits, weights) + biases)

In [10]:
num_steps = 3001

with tf.compat.v1.Session(graph=graph) as session:
  tf.compat.v1.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 329.299347
Minibatch accuracy: 7.0%
Validation accuracy: 31.0%
Minibatch loss at step 500: 31.122234
Minibatch accuracy: 75.8%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 15.380733
Minibatch accuracy: 82.0%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 6.638079
Minibatch accuracy: 79.7%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 3.309615
Minibatch accuracy: 85.9%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 2.825099
Minibatch accuracy: 85.9%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 3.383684
Minibatch accuracy: 78.9%
Validation accuracy: 82.2%
Test accuracy: 89.3%


In [11]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.compat.v1.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  hidden_layers = 1024
  
  # Variables.    
  weights = tf.Variable(
  tf.random.truncated_normal([hidden_nodes, num_labels]))
  hidden_weights = tf.Variable(tf.random.truncated_normal([image_size * image_size, hidden_nodes]))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  hidden_layers = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
  hidden_layers = tf.nn.relu(hidden_layers)
  hidden_layers=tf.nn.dropout(hidden_layers,0.4)

  logits = tf.matmul(hidden_layers,weights)+biases
  l2_regularization = lambd/(2*train_dataset.shape[0])* (tf.nn.l2_loss(weights)+tf.nn.l2_loss(hidden_weights))
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+l2_regularization

  # Optimizer.
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden = tf.matmul(tf_valid_dataset,hidden_weights)+hidden_biases
  valid_logits = tf.nn.relu(valid_hidden)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_logits, weights) + biases)
  test_hidden = tf.matmul(tf_test_dataset,hidden_weights)+hidden_biases
  test_logits = tf.nn.relu(test_hidden)
  test_prediction = tf.nn.softmax(tf.matmul(test_logits, weights) + biases)

In [12]:
num_steps = 3001

with tf.compat.v1.Session(graph=graph) as session:
  tf.compat.v1.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 437.430695
Minibatch accuracy: 10.2%
Validation accuracy: 31.5%
Minibatch loss at step 500: 43.066879
Minibatch accuracy: 71.9%
Validation accuracy: 78.7%
Minibatch loss at step 1000: 13.276115
Minibatch accuracy: 75.0%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 9.986154
Minibatch accuracy: 78.1%
Validation accuracy: 80.0%
Minibatch loss at step 2000: 3.885167
Minibatch accuracy: 83.6%
Validation accuracy: 80.8%
Minibatch loss at step 2500: 5.103954
Minibatch accuracy: 77.3%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 3.472274
Minibatch accuracy: 75.8%
Validation accuracy: 80.3%
Test accuracy: 87.6%


In [28]:
batch_size = 128
hidden_nodes1 = 1024
hidden_nodes2 = 256
hidden_nodes3 = 128
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.compat.v1.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.compat.v1.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)  # count the number of steps taken.

  
  # Variables.    
  weights = tf.Variable(tf.random.truncated_normal([hidden_nodes3, num_labels],stddev=np.sqrt(2.0 / hidden_nodes3)))
  hidden_weights1 = tf.Variable(tf.random.truncated_normal([image_size * image_size, hidden_nodes1]))
  hidden_weights2 = tf.Variable(tf.random.truncated_normal([hidden_nodes1, hidden_nodes2],stddev=np.sqrt(2.0 / hidden_nodes1)))
  hidden_weights3 = tf.Variable(tf.random.truncated_normal([hidden_nodes2, hidden_nodes3],stddev=np.sqrt(2.0 / hidden_nodes2)))
  biases = tf.Variable(tf.zeros([num_labels]))
  hidden_biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
  hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
  hidden_biases3 = tf.Variable(tf.zeros([hidden_nodes3]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  hidden_layers1 = tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1
  hidden_layers1 = tf.nn.relu(hidden_layers1)
  hidden_layers1 = tf.nn.dropout(hidden_layers1,0.5)
  hidden_layers2 = tf.matmul(hidden_layers1, hidden_weights2) + hidden_biases2
  hidden_layers2 = tf.nn.relu(hidden_layers2)
  hidden_layers2 = tf.nn.dropout(hidden_layers2,0.5)
  hidden_layers3 = tf.matmul(hidden_layers2, hidden_weights3) + hidden_biases3
  hidden_layers3 = tf.nn.relu(hidden_layers3)
  hidden_layers3 = tf.nn.dropout(hidden_layers3,0.5)

  logits = tf.matmul(hidden_layers3,weights)+biases
  l2_regularization = lambd/(2*train_dataset.shape[0])* (tf.nn.l2_loss(weights)+tf.nn.l2_loss(hidden_weights1)+\
                                                         tf.nn.l2_loss(hidden_weights2)+tf.nn.l2_loss(hidden_weights3))
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+l2_regularization

  # Optimizer.
  learning_rate = tf.compat.v1.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
  optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hidden1 = tf.matmul(tf_valid_dataset,hidden_weights1)+hidden_biases1
  valid_logits1 = tf.nn.relu(valid_hidden1)
  valid_hidden2 = tf.matmul(valid_logits1,hidden_weights2)+hidden_biases2
  valid_logits2 = tf.nn.relu(valid_hidden2)
  valid_hidden3 = tf.matmul(valid_logits2,hidden_weights3)+hidden_biases3
  valid_logits3 = tf.nn.relu(valid_hidden3)
  valid_prediction = tf.nn.softmax(tf.matmul(valid_logits3, weights) + biases)

  test_hidden1 = tf.matmul(tf_test_dataset,hidden_weights1)+hidden_biases1
  test_logits1 = tf.nn.relu(test_hidden1)
  test_hidden2 = tf.matmul(test_logits1,hidden_weights2)+hidden_biases2
  test_logits2 = tf.nn.relu(test_hidden2)
  test_hidden3 = tf.matmul(test_logits2,hidden_weights3)+hidden_biases3
  test_logits3 = tf.nn.relu(test_hidden3)
  test_prediction = tf.nn.softmax(tf.matmul(test_logits3, weights) + biases)

In [29]:
num_steps = 3001

with tf.compat.v1.Session(graph=graph) as session:
  tf.compat.v1.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 28.796667
Minibatch accuracy: 16.4%
Validation accuracy: 11.3%
Minibatch loss at step 500: nan
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 8.6%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 7.8%
Validation accuracy: 10.0%
Test accuracy: 10.0%
